In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from datetime import datetime
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix,ConfusionMatrixDisplay,roc_curve,roc_auc_score,precision_recall_curve
from sklearn.ensemble import RandomForestClassifier , StackingClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,Lasso
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_val_score,GridSearchCV

# 한글 깨짐 방지
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'malgun Gothic'

# 경고창 무시 
import warnings
warnings.filterwarnings("ignore")

In [19]:
df = pd.read_csv('./datasets/2. Labeling완료.csv', index_col=0)
df

,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,영화구분,개봉일관객수,개봉일매출액,개봉일매출비율,등급_12,등급_15,등급_전체,등급_청불,국내배급사,상장배급사,주요배급사,배우가중치,감독가중치,Label
0,1,1587,135748398910,17613682,33121225810,4163666,1,682701.0,4.708879e+09,61.6,0,1,0,0,0,1,1,39.00,2.00,0
1,1,1978,139647979516,16264944,31858660536,3638287,1,368582.0,3.004763e+09,73.2,0,1,0,0,0,1,1,21.00,3.00,1
2,1,1912,115698654137,14410754,27530825087,3346172,1,406365.0,2.984151e+09,63.2,1,0,0,0,0,1,1,27.00,3.00,0
3,1,966,110828014630,14245998,25842519330,3233946,1,184756.0,1.356870e+09,31.0,1,0,0,0,0,1,1,29.00,1.00,0
4,0,2835,122182694160,13934592,33577136860,3597963,1,1338729.0,9.678990e+09,97.1,1,0,0,0,1,0,1,22.00,3.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,0,425,2015360478,260721,722246378,90018,1,18404.0,1.352783e+08,5.6,1,0,0,0,1,0,1,0.33,0.10,1
913,1,445,2088685200,257716,411417600,50477,1,32277.0,2.502558e+08,13.1,0,0,0,1,0,0,0,0.63,0.03,0
914,0,168,2370348500,257193,880006000,87499,1,12613.0,1.180325e+08,7.5,0,0,0,1,1,0,1,0.03,0.07,1
915,0,401,1997947292,256475,605776392,75489,1,26707.0,1.993983e+08,5.8,0,1,0,0,1,0,1,0.23,0.03,0


---
# Train Test Set 분리

In [35]:
X = df.drop(columns='Label', axis=1)
y = df[['Label']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [36]:
X_train_df = X_train[['전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수', 
                   '개봉일관객수', '개봉일매출액', '개봉일매출비율', '배우가중치', '감독가중치']]
X_train_concat = X_train.drop(columns=['전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수', 
                                       '개봉일관객수', '개봉일매출액', '개봉일매출비율', '배우가중치', '감독가중치'], axis=1)

X_test_df = X_test[['전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수', 
                   '개봉일관객수', '개봉일매출액', '개봉일매출비율', '배우가중치', '감독가중치']]
X_test_concat = X_test.drop(columns=['전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수', 
                                       '개봉일관객수', '개봉일매출액', '개봉일매출비율', '배우가중치', '감독가중치'], axis=1)

In [40]:
X_train_concat.reset_index(drop=True, inplace=True)
X_test_concat.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [41]:
scaler_SS = StandardScaler()
train_scaled_SS = scaler_SS.fit_transform(X_train_df)
# transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
X_train = pd.DataFrame(data = train_scaled_SS, columns=X_train_df.columns)

test_scaled_SS = scaler_SS.transform(X_test_df)
# transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
X_test = pd.DataFrame(data = test_scaled_SS, columns=X_test_df.columns)

In [45]:
# Scaling Data와 결합

train_SS = pd.concat([X_train, X_train_concat, y_train], axis=1)
test_SS = pd.concat([X_test, X_test_concat, y_test], axis=1)

In [48]:
train_SS.to_csv('./datasets/train_ss.csv')
test_SS.to_csv('./datasets/test_ss.csv')